# Music analysis between two cities

## Introduction <a id='intro'></a>

In this project we will be comparing the musical preferences of Springfield and Shelbyville. We will use real data of music streaming to prove a hypothesis and compare the behavior of the users of these two cities.

### Objective:

The following hypothesis will be tested:

* User activity varies according to the day of the week and depending on the city.


## Data description <a id='data_review'></a>

Read and analyze our data

In [1]:
# Import pandas
import pandas as pd

In [2]:
# Read the file and store it in the dataframe
df = pd.read_csv(r"C:\Users\marco\Documents\tripleten_projects\final_projects\sprint2\music_project_en.csv")

In [3]:
# Extract the first 10 first rows from the dataframe
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


In [4]:
# Obtain general information about our data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


### Conclusions <a id='#data_review_conclusions'></a>

We observe that it is a table with 7 columns, in which all the data has been stored as `'object'` despite containing different types of data.

According to the documentation:
- 'userID': user identifier;
- 'Track': song title;
- 'artist': artist name;
- 'genre': track genre;
- 'City': user's city;
- 'time': the exact time the song was played;
- 'Day': day of the week.

We can see three problems with the style in the table headers:
1. Some headers are in uppercase, others in lowercase.
2. There are blank spaces in some headers.
3. The use of snake case is missing in the name of the first column.



#### Data Processing Observations

* Within the first 10 rows, in the `'userID'` column, not all displayed data have the same length in an alphanumeric format.
* In the tenth row, a missing value is observed in the `'artist'` column.
* It will be necessary to review the 'artist' column (if feasible given the amount of data) for spelling inconsistencies in the names of the artists.
* The number of unique values in the 'City' column will need to be reviewed, as there should be only 2.
* The data format in the time column is in the form of H:mm:ss.
* Unique values in the 'Day' column need to be reviewed to verify that there are only 7 values.
* It is observed that there are missing data in the `'Track'`, `'artist'`, and `'genre'` columns.
* There is considered to be sufficient information to answer the hypothesis.

## Data preprocessing<a id='data_preprocessing'></a>


### Header Style <a id='header_style'></a>


In [5]:
# The column names are displayed
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

The table headers are changed to follow best practices:

* Letters should be lowercase.
* Remove blank spaces.
* Use snake_case.

In [6]:
# Loop to change headers to lowercase

# A list is created to add the names of the columns once modified
lower_col_names = []

# A loop is created to work with each column of the dataframe
for old_name in df.columns: 
    name_lower=old_name.lower() # The name format is changed to lowercase and saved as name_lower
    lower_col_names.append(name_lower) # The changed name is added to the lower_col_names list

# The column names are changed to those in the lower_col_names list
df.columns=lower_col_names 
df.columns 

Index(['  userid', 'track', 'artist', 'genre', '  city  ', 'time', 'day'], dtype='object')

In [7]:
# Loop to remove spaces in the headers

# A list is created to add the names of the columns once modified
stripped_col_names = [] 

# A loop is created to work with each column of the dataframe
for old_name in df.columns: 
    name_stripped=old_name.strip() # Spaces before and after the name are trimmed, and saved as name_stripped
    stripped_col_names.append(name_stripped) # The changed name is added to the stripped_col_names list

# The column names are changed to those in the stripped_col_names list
df.columns=stripped_col_names 
df.columns 

Index(['userid', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

In [8]:
# Change the column name 'userid' to snake case

# The dataframe is named using the rename() method, indicating that the 'userid' column is to be changed and replaced with 'user_id'
df.rename(columns={'userid':'user_id'}, inplace=True) 
                                                    
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

In [9]:
# Verify the change in the list of headers
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Missing values <a id='missing_values'></a>


In [10]:
# Calculate the number of missing values
print(df.isna().sum())

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64


Not all missing values affect the investigation. For example, missing values in `'track' and 'artist'` are not relevant, so they will be assigned as `'unknown'`.

Missing values in `'genre'` can affect the comparison between the musical preferences of Springfield and Shelbyville. So we will do the following:

* Fill in missing values with a default value;
* Evaluate how much missing values could affect

In [11]:
# Loop through headers replacing missing values with 'unknown'

# A list is made with the name of the columns to work with
columns_to_replace=['track','artist','genre']

# A loop is made to work with each column in the list
for col in columns_to_replace: 
    df[col].fillna('unknown',inplace=True) # Replace null values within the columns with 'unknown'
                                           

In [12]:
# We check that we no longer have missing values
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Duplicates <a id='duplicates'></a>

In [13]:
# We count the explicit duplicates
print(df.duplicated().sum())

3826


In [14]:
# We remove the explicit duplicates
df.drop_duplicates(inplace=True)

In [15]:
# We check again for duplicates
print(df.duplicated().sum())

0


Now, we want to get rid of implicit duplicates in the 'genre' column. For example, the name of a genre can be written in several ways. Such errors can also affect the result.

To do this, first let's display a list of unique genre names, sorted in alphabetical order. To achieve this:

* Extract the `'genre'` column from the DataFrame.
* Call the method that will return all unique values in the extracted column.

In [16]:
# Review the unique genre names
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

An example in the list to find implicit duplicates is the genre `hiphop`. These could be misspelled names or alternative names for the same genre.

The following implicit duplicates are observed:

* `hip`
* `hop`
* `hip-hop`

To get rid of them, we will create a function called `replace_wrong_genres()` with two parameters:

* `wrong_genres=`: list containing all the values we need to replace.
* `correct_genre=`: string we will use as replacement.

As a result, the function should correct the names in the `'genre'` column of the df table, i.e., replace each value from the `wrong_genres` list with the value in `correct_genre`

In [17]:
# Function to replace implicit duplicates


def replace_wrong_genres(df,column,wrong_genres,correct_genre): 
    
    # A loop is created to work with each genre from the list of incorrect genres
    for wrong_genre in wrong_genres: 
        
    # The change is made in the genre name in the column
        df[column]=df[column].replace(wrong_genre,correct_genre) 
        
    # Returns the updated DataFrame as a result   
    return df 

In [18]:
# Remove implicit duplicates

# A list is made with the name of the incorrect genres
duplicates = ['hip','hop','hip-hop'] 

# The correct name is defined in a variable
correct = 'hiphop' 

# The DataFrame is updated with the function that was performed
df = replace_wrong_genres(df,'genre',duplicates,correct) 

In [19]:
# The correction is checked
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

### Observations <a id='data_preprocessing_conclusions'></a>

By removing the duplicates, we ensured that we did not make a flawed comparison when testing our hypothesis. We used the `drop_duplicates()` method to remove explicit duplicates and created a function to iterate over our data and replace the names of incorrect genres, in this case, `hip hop`. Now that we have finished reviewing and cleaning our data, we can proceed with our analysis.

## Hypothesis test <a id='hypothesis'></a>

### Hypothesis: compare the user behavior in both cities <a id='activity'></a>

The hypothesis states that there are differences in how users in Springfield and Shelbyville consume music. To test this, we will use data from three days of the week: Monday, Wednesday, and Friday.

* We will group users by city.
* We will compare the number of songs each group played on Monday, Wednesday, and Friday

In [20]:
# Counting the songs played in each city
df.groupby(by='city')['track'].count()

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64

It is observed that in Springfield there is a tendency to play 2.3 times more songs than in Shelbyville.

In [21]:
# Calculate the songs played on each of the three days
df.groupby(by='day')['track'].count().sort_values()

day
Wednesday    18059
Monday       21354
Friday       21840
Name: track, dtype: int64

It is observed that the day with the fewest plays is Wednesday. The day with the most plays is Friday, and Monday comes in second place, with not much difference from Friday. We could deduce that users listen to more music on these days, Monday because it's the start of the week, and Friday because the weekend is about to begin.

We create a function to obtain the number of plays per day and city, using these two as parameters of the function

In [22]:
# The function number_tracks() is declared with two parameters: day and city
def number_tracks(day,city): 
    
    # The rows are grouped by 'day'
    filt_df=df[df['day'] == day]
    # The filt_df is updated filtered by 'city'
    filt_df=filt_df[filt_df['city']==city]
    # The 'user_id' column is extracted and count() is applied
    times_played=filt_df['user_id'].count()
    # Return the number of values
    return times_played

In [23]:
print('The number of songs played in Springfield on Monday:',number_tracks('Monday','Springfield'))

The number of songs played in Springfield on Monday: 15740


In [24]:
print('The number of songs played in Shelbyville on Monday:',number_tracks('Monday','Shelbyville'))

The number of songs played in Shelbyville on Monday: 5614


In [25]:
print('The number of songs played in Springfield on Wednesday:',number_tracks('Wednesday','Springfield'))

The number of songs played in Springfield on Wednesday: 11056


In [26]:
print('The number of songs played in Shelbyville on Wednesday:',number_tracks('Wednesday','Shelbyville'))

The number of songs played in Shelbyville on Wednesday: 7003


In [27]:
print('The number of songs played in Springfield on Friday:',number_tracks('Friday','Springfield'))

The number of songs played in Springfield on Friday: 15945


In [28]:
print('The number of songs played in Shelbyville on Friday:',number_tracks('Friday','Shelbyville'))

The number of songs played in Shelbyville on Friday: 5895


### Hypothesis Conclusion



The hypothesis worked on is correct. There are noticeable differences between the cities in the number of songs listened to, and a difference between Wednesday compared to Monday and Friday. Wednesday being the highest day in Shelbyville, and Monday and Friday in Springfield.

# Conclusions <a id='end'></a>

The conclusions can be observed from three different points of view:

**1. City vs City** 
We can observe that when comparing cities, Springfield listens to 2.4 times more on average over the three days studied. We could study the social contexts of each city to discover the reason for this.

**2. Day vs Day** 
It is observed that although Monday and Friday are close in figures, there is a decrease in songs played on Wednesday. We could think that since it is the beginning of the week and the beginning of the weekend, these are days when there are mood changes due to what these days represent. Knowledge in this area could be amplified if we had the ages of the sampled users and saw the age distribution, or if we knew their current employment or student status, as these days may represent the beginning or end of work/student time in the week.

**3. City-Day vs. City-Day**
A different trend is observed in the days with the highest listening in each city. Monday and Friday are the highest in Springfield, as Wednesday in Shelbyville. Using information from the entire week could provide more insights along with the ages of the users, to see their influence on listening time depending on the dominant age group in each city.